## 필수 라이브러리 가져오기  
Phi-4 모델을 로드하고 사용하기 위해 필요한 PyTorch와 transformers 라이브러리를 가져옵니다.  


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## 랜덤 시드 설정
랜덤 시드를 설정하여 다른 실행 간에 재현 가능한 결과를 보장합니다.


In [ ]:
torch.random.manual_seed(0)

## Phi-4-mini-flash-reasoning 모델과 토크나이저 로드하기
Microsoft의 Phi-4-mini-flash-reasoning 모델과 해당 토크나이저를 Hugging Face에서 로드합니다. 모델은 더 빠른 추론을 위해 CUDA에 로드됩니다.


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## 입력 메시지 준비하기
모델의 채팅 템플릿을 사용하여 이차 방정식 수학 문제를 포함한 대화 메시지를 작성하세요.


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## 응답 생성
출력의 무작위성을 조절하기 위해 temperature와 top_p 같은 매개변수를 사용하여 모델의 응답을 생성합니다.


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## 출력 디코딩하여 텍스트로 변환하기
생성된 토큰 시퀀스를 인간이 읽을 수 있는 텍스트로 변환합니다. 원래 입력 토큰을 제외하고 모델의 응답만 표시합니다.


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
